În partea precedentă am aflat informații teoretice despre cum funcționează procesul de Feature Extraction from Text, iar acuma o să realizăm acest procedeu utilizând limbaj de Python, o să creem manual acest procedeu de extracție de informații din text. În partea a doua o să folosim aceste concepte cu Scikit-Learn care poate face automat această treabă. O să lucrăm cu două fișiere de tip text pentru a putea realiza asta.

In [1]:
with open('../data/18-Naive-Bayes-and-NLP/One.txt') as file:
    a = file.read()

In [2]:
a

'This is a story about dogs\nour canine pets\nDogs are furry animals\n'

După cum se poate oberva am citit datele din fișier, însă acestea au fost citite sub formă de un singur string, din acest motiv avem și acele caractere care ne sugerează final de linie (\n). Dacă utilizăm print, atunci o să dispară acele caractere.

In [3]:
print(a)

This is a story about dogs
our canine pets
Dogs are furry animals



Din moment ce am zis că acest algoritm are de numărat fiecare cuvânt, ar fi ideal să avem aceste cuvite într-o listă separată

In [4]:
a.lower().split()

['this',
 'is',
 'a',
 'story',
 'about',
 'dogs',
 'our',
 'canine',
 'pets',
 'dogs',
 'are',
 'furry',
 'animals']

În continuare o să creem un vocabular de cuvinte, vocabular care are o importanță extrem de mare în ceea ce privește Natural Language Processing. Acest dicționar o să conțină cuvintele unicare care apar în toate documentele. Pentru acest caz o să lucrăm cu două fișiere, One.txt, respectiv Two.txt și o să creem un set de valori unicate cu elementele din fiecare document.

In [5]:
with open('../data/18-Naive-Bayes-and-NLP/One.txt') as my_text:
    words_one = my_text.read().lower().split()
    uni_words_one = set(words_one)

In [23]:
with open('../data/18-Naive-Bayes-and-NLP/Two.txt') as my_text:
    words_two = my_text.read().lower().split()
    uni_words_two = set(words_two)

In [24]:
uni_words_one

{'a',
 'about',
 'animals',
 'are',
 'canine',
 'dogs',
 'furry',
 'is',
 'our',
 'pets',
 'story',
 'this'}

In [25]:
uni_words_two

{'a',
 'about',
 'catching',
 'fun',
 'is',
 'popular',
 'sport',
 'story',
 'surfing',
 'this',
 'water',
 'waves'}

În acest moment avem câte un set de cuvinte pentru fiecare document în parte, ceea ce trebuie să facem este să creem un set care să conțină valorile unice din aceste două documente. Pentru asta o să utilizăm tot seturi.

In [26]:
all_uni_words = set()
all_uni_words.update(uni_words_one)
all_uni_words.update(uni_words_two)
all_uni_words

{'a',
 'about',
 'animals',
 'are',
 'canine',
 'catching',
 'dogs',
 'fun',
 'furry',
 'is',
 'our',
 'pets',
 'popular',
 'sport',
 'story',
 'surfing',
 'this',
 'water',
 'waves'}

Următorul pas este să atribuim fiecărui cuvânt câte un număr pentru a avea o referință la acele numere.

In [27]:
vocabulary = dict()

In [28]:
i = 0

for word in all_uni_words:
    vocabulary[word]= i
    i = i + 1

In [29]:
vocabulary

{'this': 0,
 'about': 1,
 'our': 2,
 'fun': 3,
 'is': 4,
 'story': 5,
 'water': 6,
 'waves': 7,
 'catching': 8,
 'dogs': 9,
 'sport': 10,
 'animals': 11,
 'popular': 12,
 'a': 13,
 'furry': 14,
 'are': 15,
 'canine': 16,
 'pets': 17,
 'surfing': 18}

Prin această modalitate acuma avem câte un identificator unic pentru fiecare cuvânt din acel set. În partea care urmează trebuie să creem trei liste separate, fiecare dintre ele având atâtea elemente câte elemente sunt în cadrul dicționarului respectiv

In [30]:
one_freq = [0] * len(vocabulary)
two_freq = [0] * len(vocabulary)
all_freq = [''] * len(vocabulary)

Listele de mai sus o să conțină valori care fac referire la fiecare apariție a cuvintelor din vocabular în cadrul unui document specific. De exemplu, lista 'one_freq' o să conțină numere care reprezintă de câte ori a apărut un anumit cuvânt în primul document. Lista all_freq o să conțină o lista cuvintelor din toate documentele aranjate după cum apar în cele două liste unde se verifică frecvența cuvintelor.

In [31]:
one_freq

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [32]:
with open('../data/18-Naive-Bayes-and-NLP/One.txt') as file_one:
    words_one = file_one.read().lower().split()

for word in words_one:
    word_index = vocabulary[word]
    one_freq[word_index] += 1

In [33]:
one_freq

[1, 1, 1, 0, 1, 1, 0, 0, 0, 2, 0, 1, 0, 1, 1, 1, 1, 1, 0]

In [39]:
vocabulary

{'this': 0,
 'about': 1,
 'our': 2,
 'fun': 3,
 'is': 4,
 'story': 5,
 'water': 6,
 'waves': 7,
 'catching': 8,
 'dogs': 9,
 'sport': 10,
 'animals': 11,
 'popular': 12,
 'a': 13,
 'furry': 14,
 'are': 15,
 'canine': 16,
 'pets': 17,
 'surfing': 18}

După cum se poate observa, în cadrul listei one_freq avem valori numerice de la 0 la 2. O valoare reprezintă de câte ori a apărut cuvântul din dicționar în acel document. De exemplu, cuvântul de pe index-ul 0 din dicționar îi corespunde valoarea de pe index-ul 0 din lista one_freq, prin urmare, cuvântul 'this' apare odată în primul document, cuvântul 'about' apare tot odată și tot așa. Același procedeu o să îl facem și pentru al doilea document

In [34]:
with open('../data/18-Naive-Bayes-and-NLP/Two.txt') as file_two:
    words_two = file_two.read().lower().split()

for word in words_two:
    word_index = vocabulary[word]
    two_freq[word_index] += 1

In [35]:
two_freq

[1, 1, 0, 1, 3, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 2]

Am făcut același lucru și cu cel de-al doilea document. Urmează să creem o listă cu toate cunvintele în ordinea în care apar acestea în dicționar. Această listă ne ajută la crearea unui DataFrame pentru vizualizare.

In [36]:
for word in vocabulary:
    word_index = vocabulary[word]
    all_freq[word_index] = word

In [37]:
all_freq

['this',
 'about',
 'our',
 'fun',
 'is',
 'story',
 'water',
 'waves',
 'catching',
 'dogs',
 'sport',
 'animals',
 'popular',
 'a',
 'furry',
 'are',
 'canine',
 'pets',
 'surfing']

După ce avem și această listă putem să creem acel DataFrame pentru a vizualiza mai bine informațiile.

In [38]:
import pandas as pd
pd.DataFrame(data=[one_freq, two_freq], columns=all_freq)

,this,about,our,fun,is,story,water,waves,catching,dogs,sport,animals,popular,a,furry,are,canine,pets,surfing
0,1,1,1,0,1,1,0,0,0,2,0,1,0,1,1,1,1,1,0
1,1,1,0,1,3,1,1,1,1,0,1,0,1,1,0,0,0,0,2


Ceea ce am creat mai sus poartă denumirea de 'bag of words model' care are scopul de a calcula frecvența tuturor cuvintelor din documente. Valorile pentru fiecare cuvânt în parte pot fi asemănătoare pentru toate documentele sau pot fi extrem de diferite. Prin acest procedeu am modificat text în valori numerice pe care putem să le oferim unui model de Machine Learning. 

Datele de mai sus nu sunt chiar așa de folositoare până ce nu introducem și conceptele de frecvența cuvintelor și cât de des apar aceste cuvinte în anumite documente. Dacă avem de-a face cu documente care se găsesc în același domeniu, de exemplu, toate documentele fac parte din categoria sport și încercăm să clasificăm documentele în sporturi (baschet, fotbal, tenis) există anumite cuvinte care apar în toate documentele, cum ar fi alergare sau minge, sau cuvinte care sunt specifice pentru un singur sport (coș la baschet, sau gol la fotbal). Pentru a face diferența dintre documente trebuie să luăm în calcul și acești factori.

În continuare o să creem acest algoritm cu Scikit-Learn, iar acesta ne permite să realizăm acest pas în doar câteva linii de cod.